In [1]:
import SimpleITK as sitk   # pip install SimpleITK
import numpy as np     # pip install numpy
import matplotlib.pyplot as plt
import pydicom         # pip install pydicom
import cv2          # pip install opencv-python
import os 

In [2]:
def readDCM(path):
    dcm = pydicom.dcmread(path)
    # image_array = np.squeeze(sitk.GetArrayFromImage(dcm))
    return dcm

def readMHA(path):
    mha = sitk.ReadImage(path)
    mha = sitk.GetArrayFromImage(mha)
    return mha

In [3]:
image_seg = readMHA('./Abdominal_Data/Reference/segmentation.mha')
image_seg = image_seg.reshape(256,256)

In [4]:
FilePath= "./Abdominal_Data/Moving"  
FilePath2= "./Abdominal_Data/Reference"
NewMovingPath = './JPG'  # build a new file to save images

def DCMto_mha(filepath):
    '''
    convert .dcm format to .mha and store in the same path
    '''
    
    lstFilesDCM = []
    for dirName, subdirList, fileList in os.walk(filepath):
        for filename in fileList:
            if ".dcm" in filename.lower():  
                lstFilesDCM.append(os.path.join(dirName, filename))  

    for i in range(len(lstFilesDCM)):
        RefDs = pydicom.read_file(lstFilesDCM[i]) 
        ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns)) 
        ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]))
        Origin = RefDs.ImagePositionPatient

        ArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)  # array is a numpy array
        ArrayDicom[:,:] = RefDs.pixel_array 

        sitk_img = sitk.GetImageFromArray(ArrayDicom, isVector=False)
        sitk_img.SetSpacing(ConstPixelSpacing)
        sitk_img.SetOrigin(Origin)
        sitk.WriteImage(sitk_img, os.path.join(lstFilesDCM[i].replace('.dcm', '.mha')))
        
DCMto_mha(FilePath)
DCMto_mha(FilePath2)

In [5]:
def Segmentation(filepath):
    '''
    identify the various parts of image and extract the left lung as a result
    return: a binary matrix of pixels which compose the left lung
    '''
    
    reader = sitk.ImageFileReader()
    reader.SetFileName(filepath)
    image = reader.Execute()
    
    
    # Preprocess image: Blur image using CurvatureFlowImageFilter
    blurFilter = sitk.CurvatureFlowImageFilter()
    blurFilter.SetNumberOfIterations(5)
    blurFilter.SetTimeStep(0.125)
    image = blurFilter.Execute(image)

    # Segmentation code
    segmentationFilter = sitk.ConnectedThresholdImageFilter()
    segmentationFilter.SetLower(0)
    segmentationFilter.SetUpper(120)
    segmentationFilter.SetReplaceValue(255)  

    seed = [60, 120]
    segmentationFilter.AddSeed(seed)
    image = segmentationFilter.Execute(image) 

    writer = sitk.ImageFileWriter()
    writer.SetFileName(filepath)
    writer.Execute(image)

    image = sitk.GetArrayFromImage(image)
    return image.reshape(256,256)

binary_img = Segmentation('./Abdominal_Data/Reference/refLung_001.mha')

In [6]:
index = np.argwhere(image_seg == 1)
minrange, maxrange= index.min(axis=0), index.max(axis=0)
seg_filter = image_seg[minrange[0]:maxrange[0]+1, minrange[1]:maxrange[1]+1]

In [7]:
feature_seg = (image_seg * binary_img)[minrange[0]:maxrange[0]+1, minrange[1]:maxrange[1]+1]
feature_mean = feature_seg.mean()
feature_std = feature_seg.std()

In [8]:
def Get_segmentation(image, seg_filter):
    '''
    to analyze the image so that determine the target position
    param:
        image: the matrix array of pixels
        seg_filter: the matrix array of annotation which is from "segmentation.mha"
    return:
        mincentroid: the position where the seg_filter(annotation) should be
        minMatrix: the matrix of pixels with seg_filter(annotation)
    '''
    rows, cols = image.shape
    mask_rows, mask_cols = seg_filter.shape

    centroid = (int(mask_rows/2), int(mask_cols/2))
    
    handle_matrix = image.copy()
    
    feature_init = feature_mean+feature_std
    centroid_init = (minrange+maxrange)//2
    
    minfeature = float('inf')
    mincentroid = [0, 0]
    minMatrix = []
    
    # compute the Characteristic Value and then determine the position of annotation
    for row in range(centroid_init[0]-15, centroid_init[0]+15):
        for col in range(centroid_init[1]-10, centroid_init[1]+10):
            matrix = image[row-centroid[0]:row-centroid[0]+mask_rows,col-centroid[1]:col-centroid[1]+mask_cols] * seg_filter
            feature = matrix.mean() + matrix.std() + (sum((centroid_init-np.array([row, col]))**2))*0.9
            
            feature = abs(feature-feature_init)
            if feature < minfeature:
                minfeature = feature
                mincentroid = [row, col]
                minMatrix = image[row-centroid[0]:row-centroid[0]+mask_rows,col-centroid[1]:col-centroid[1]+mask_cols].copy()
    
    
    return mincentroid, minMatrix

In [9]:
def RestoreImage(image, seg_filter, center):
    '''
    based on the target position to generate a image with annotation of target
    param:
        image: the matrix array of pixels
        seg_filter: the matrix array of annotation which is from "segmentation.mha"
        center: the position where the seg_filter should be on the image
        
    return: a matrix of rgb image with annotation
        
    '''
    rows, cols = image.shape
    mask_rows, mask_cols = seg_filter.shape
    temp_image = image.copy()
    
    centerx, centery = center
    centerx=centerx-int(mask_rows/2)
    centery=centery-int(mask_cols/2)
    updateFilter = seg_filter.copy()
    updateFilter[updateFilter==1] = 1
    updateFilter[updateFilter==0] = 0
    new_updateFilter = image.copy()
    new_updateFilter[new_updateFilter!=-1] = 0
    new_updateFilter[centerx:centerx+mask_rows, centery:centery+mask_cols] += updateFilter
    
    temp_image[centerx:centerx+mask_rows, centery:centery+mask_cols] = temp_image[centerx:centerx+mask_rows, centery:centery+mask_cols] + updateFilter
    temp = np.full((rows,cols,3), np.array([0,0,0]))
    temp = np.asarray(temp, np.float64)
    
    temp[new_updateFilter==1] += np.array([0, 0, 255])
    
    image_rgb = np.float32(temp_image)
    image_rgb = cv2.cvtColor(image_rgb, cv2.COLOR_GRAY2BGR)
    image_rgb = np.int32(image_rgb)
    temp = np.int32(temp)
    
    result = cv2.addWeighted(image_rgb,0.2,temp,0.3,0)

    return result
    
    
    

In [10]:
def get_frame(path):
    '''
    save the annotated image as .jpg format which can be generated a video
    param:
        path: the path of directory of images
    '''
    img_seg = Segmentation(path)
    img_orin = readDCM(path.replace('.mha','.dcm'))
    img_orin = img_orin.pixel_array
    
    
    centroid, matrix = Get_segmentation(img_seg, seg_filter)
    frame_matrix = RestoreImage(img_orin, seg_filter, centroid)
    
    cv2.imwrite(path.replace('.mha','.jpg'), frame_matrix)
    

In [11]:
def delete_intermediate(path):
    '''
    there are many intermediate image generated. the function is used to delete them
    param: 
        path: the path of directory of images
    '''
    g = os.walk(FilePath)
    for path, dir_list, file_list in g:
        for file_name in file_list:
            if os.path.splitext(file_name)[1] == ".mha":
                os.remove(os.path.join(path, file_name))
            if os.path.splitext(file_name)[1] == ".jpg":
                os.remove(os.path.join(path, file_name))

In [12]:
g = os.walk(FilePath)
for path, dir_list, file_list in g:
    for file_name in file_list:
        if os.path.splitext(file_name)[1] == ".mha":
            get_frame(os.path.join(path, file_name))

In [13]:
fps = 2    
size=(256,256)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
videoWriter = cv2.VideoWriter('./result.avi',fourcc,fps,size)

g = os.walk(FilePath)

for path, dir_list, file_list in g:
    for file_name in file_list:
        if os.path.splitext(file_name)[1] == ".jpg":
            frame = cv2.imread(os.path.join(path, file_name))
            videoWriter.write(frame)
videoWriter.release()
delete_intermediate(FilePath)